step 1: add the special token to the vocabulary

In [63]:
vocab = {}

special_token = "<|endoftext|>"
vocab.update({special_token: 0})


step 2: add 256 byte values to the vocabulary

In [64]:
for i in range(256):
    char = chr(i)
    vocab.update({char: i + 1})

In [65]:
vocab

{'<|endoftext|>': 0,
 '\x00': 1,
 '\x01': 2,
 '\x02': 3,
 '\x03': 4,
 '\x04': 5,
 '\x05': 6,
 '\x06': 7,
 '\x07': 8,
 '\x08': 9,
 '\t': 10,
 '\n': 11,
 '\x0b': 12,
 '\x0c': 13,
 '\r': 14,
 '\x0e': 15,
 '\x0f': 16,
 '\x10': 17,
 '\x11': 18,
 '\x12': 19,
 '\x13': 20,
 '\x14': 21,
 '\x15': 22,
 '\x16': 23,
 '\x17': 24,
 '\x18': 25,
 '\x19': 26,
 '\x1a': 27,
 '\x1b': 28,
 '\x1c': 29,
 '\x1d': 30,
 '\x1e': 31,
 '\x1f': 32,
 ' ': 33,
 '!': 34,
 '"': 35,
 '#': 36,
 '$': 37,
 '%': 38,
 '&': 39,
 "'": 40,
 '(': 41,
 ')': 42,
 '*': 43,
 '+': 44,
 ',': 45,
 '-': 46,
 '.': 47,
 '/': 48,
 '0': 49,
 '1': 50,
 '2': 51,
 '3': 52,
 '4': 53,
 '5': 54,
 '6': 55,
 '7': 56,
 '8': 57,
 '9': 58,
 ':': 59,
 ';': 60,
 '<': 61,
 '=': 62,
 '>': 63,
 '?': 64,
 '@': 65,
 'A': 66,
 'B': 67,
 'C': 68,
 'D': 69,
 'E': 70,
 'F': 71,
 'G': 72,
 'H': 73,
 'I': 74,
 'J': 75,
 'K': 76,
 'L': 77,
 'M': 78,
 'N': 79,
 'O': 80,
 'P': 81,
 'Q': 82,
 'R': 83,
 'S': 84,
 'T': 85,
 'U': 86,
 'V': 87,
 'W': 88,
 'X': 89,
 'Y': 90

step 3: pretokenization

In [66]:
text = """low low low low low
lower lower widest widest widest
newest newest newest newest newest newest"""


words = text.split()
print(words)
from collections import Counter
word_counts = Counter(words)
frequency_table = dict(word_counts)

frequency_table = {tuple(word): count for word, count in frequency_table.items()}
frequency_table

['low', 'low', 'low', 'low', 'low', 'lower', 'lower', 'widest', 'widest', 'widest', 'newest', 'newest', 'newest', 'newest', 'newest', 'newest']


{('l', 'o', 'w'): 5,
 ('l', 'o', 'w', 'e', 'r'): 2,
 ('w', 'i', 'd', 'e', 's', 't'): 3,
 ('n', 'e', 'w', 'e', 's', 't'): 6}

step4: merge

In [42]:
#  count the frequency of all adjacent character pairs

merge_tables = {}
for word, count in frequency_table.items():
   
    for i in range(len(word)-1):
     
        char_pair = word[i:i+2]
        if char_pair not in merge_tables:
            merge_tables[char_pair] = count
        else:
            merge_tables[char_pair] += count
    
merge_tables


{('l', 'o'): 7,
 ('o', 'w'): 7,
 ('w', 'e'): 8,
 ('e', 'r'): 2,
 ('w', 'i'): 3,
 ('i', 'd'): 3,
 ('d', 'e'): 3,
 ('e', 's'): 9,
 ('s', 't'): 9,
 ('n', 'e'): 6,
 ('e', 'w'): 6}

In [47]:
# merge the most frequent pair
most_frequent_pair = max(merge_tables, key=merge_tables.get)
print("Most frequent pair:", most_frequent_pair, "with count:", merge_tables[most_frequent_pair])

# update the frequency table by merging the most frequent pair
new_frequency_table = {}
for word, count in frequency_table.items():
    new_word = []
    i = 0
    while i < len(word):
        if i < len(word) - 1 and word[i:i+2] == most_frequent_pair:
            new_word.append(''.join(most_frequent_pair))
            i += 2
        else:
            new_word.append(word[i])
            i += 1
    new_frequency_table[tuple(new_word)] = count


new_frequency_table

Most frequent pair: ('e', 's') with count: 9


{('l', 'o', 'w'): 5,
 ('l', 'o', 'w', 'e', 'r'): 2,
 ('w', 'i', 'd', 'es', 't'): 3,
 ('n', 'e', 'w', 'es', 't'): 6}

In [49]:
#  count the frequency of all adjacent character pairs
merge_tables = {}
for word, count in new_frequency_table.items():
    
     for i in range(len(word)-1):
      
          char_pair = word[i:i+2]
          if char_pair not in merge_tables:
                merge_tables[char_pair] = count
          else:
                merge_tables[char_pair] += count

merge_tables

{('l', 'o'): 7,
 ('o', 'w'): 7,
 ('w', 'e'): 2,
 ('e', 'r'): 2,
 ('w', 'i'): 3,
 ('i', 'd'): 3,
 ('d', 'es'): 3,
 ('es', 't'): 9,
 ('n', 'e'): 6,
 ('e', 'w'): 6,
 ('w', 'es'): 6}

In [50]:
# merge the most frequent pair
most_frequent_pair = max(merge_tables, key=merge_tables.get)
print("Most frequent pair:", most_frequent_pair, "with count:", merge_tables[most_frequent_pair])

# update the frequency table by merging the most frequent pair
frequency_table = {}
for word, count in new_frequency_table.items():
    new_word = []
    i = 0
    while i < len(word):
        if i < len(word) - 1 and word[i:i+2] == most_frequent_pair:
            new_word.append(''.join(most_frequent_pair))
            i += 2
        else:
            new_word.append(word[i])
            i += 1
    frequency_table[tuple(new_word)] = count


frequency_table

Most frequent pair: ('es', 't') with count: 9


{('l', 'o', 'w'): 5,
 ('l', 'o', 'w', 'e', 'r'): 2,
 ('w', 'i', 'd', 'est'): 3,
 ('n', 'e', 'w', 'est'): 6}

将merge这个过程写成loop

In [80]:
def merge(merge_counts):
    current_count = 1
    while current_count <= merge_counts:
        print("\nMerge iteration:", current_count)
        #  count the frequency of all adjacent character pairs
        merge_tables = {}
        for word, count in frequency_table.items():
            
             for i in range(len(word)-1):
              
                  char_pair = word[i:i+2]
                  if char_pair not in merge_tables:
                        merge_tables[char_pair] = count
                  else:
                        merge_tables[char_pair] += count

        # merge the most frequent pair
        most_frequent_pair = max(merge_tables, key=merge_tables.get)
        print("Most frequent pair:", most_frequent_pair, "with count:", merge_tables[most_frequent_pair])

        # update the frequency table by merging the most frequent pair
        new_frequency_table = {}
        for word, count in frequency_table.items():
            new_word = []
            i = 0
            while i < len(word):
                if i < len(word) - 1 and word[i:i+2] == most_frequent_pair:
                    new_word.append(''.join(most_frequent_pair))
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_frequency_table[tuple(new_word)] = count

        frequency_table.clear()
        frequency_table.update(new_frequency_table)
        current_count += 1 

    return frequency_table, merge_tables

In [81]:
frequency_table, merge_tables = merge(5)


Merge iteration: 1
Most frequent pair: ('lo', 'w') with count: 7

Merge iteration: 2
Most frequent pair: ('n', 'e') with count: 6

Merge iteration: 3
Most frequent pair: ('ne', 'w') with count: 6

Merge iteration: 4
Most frequent pair: ('new', 'est') with count: 6

Merge iteration: 5
Most frequent pair: ('w', 'i') with count: 3


In [82]:
merge_tables

{('low', 'e'): 2, ('e', 'r'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est'): 3}

In [83]:
frequency_table

{('low',): 5, ('low', 'e', 'r'): 2, ('wi', 'd', 'est'): 3, ('newest',): 6}